In [117]:
import pandas as pd 
import os 

In [118]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("preprocessing").getOrCreate()

In [133]:
directory = f"{os.getcwd()}/data"
print(directory)
sg = "gyoung.csv"
bus = "bus.csv"

sg_data = spark.read.csv(f"{directory}/{sg}", inferSchema=True, header=True)
bus_data = spark.read.csv(f"{directory}/{bus}", inferSchema=True, header=True)

sg_data = sg_data.createOrReplaceTempView("sg_data")
bus_data = bus_data.createOrReplaceTempView("bus_data")

/Users/imhaneul/Documents/project/BusStopPlace/SparkPreprocessing/analysis/data


In [146]:
# """
# /Users/imhaneul/Documents/project/BusStopPlace/SparkPreprocessing/analysis/data
# root
#  |-- 정류장아이디: string (nullable = true)
#  |-- 정류장 명칭: string (nullable = true)
#  |-- 정류장 유형: double (nullable = true)
#  |-- 중앙차로 여부: string (nullable = true)
#  |-- 노드영문명: string (nullable = true)
#  |-- 위도: double (nullable = true)
#  |-- 경도: double (nullable = true)
#  |-- 수집일시: string (nullable = true)
#  |-- 단축아이디: double (nullable = true)
#  |-- 도시코드: integer (nullable = true)
#  |-- 도시명: string (nullable = true)
#  |-- _c0: string (nullable = true)
#     bus_data.`정류장아이디` as id,
#     bus_data.`정류장 명칭`, as name,
#     bus_data._c0 as location
#"""
qs = """
SELECT 
    `정류장아이디`,
    `정류장 명칭`,
    `위도`,
    `경도`,
    `수집일시`,
    `단축아이디`,
    `도시코드`,
    sg_data.`도시명`
FROM 
    bus_data
    JOIN
        sg_data
    ON
        bus_data.`도시명` = sg_data.`도시명`
"""
sg = spark.sql(qs).toPandas()

In [150]:
sg.to_csv("test.csv", index=False, index_label=False)

'도시명'